# Libraries and Google Drive

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

from owslib.wms import WebMapService
import matplotlib.pyplot as plt

from PIL import Image
from IPython.display import display
from IPython.display import Image as displayImage

import io
from io import BytesIO
import time
import random
import os

import rasterio
from rasterio.transform import from_bounds
from rasterio.plot import show
from rasterio.crs import CRS
import rasterio.features
from rasterio.features import geometry_mask

import shapely
from shapely.geometry import Polygon, LineString

from pyproj import Geod
from pyproj import Transformer
from geopy.distance import distance

import math

import ast

In [107]:
# Mount Drive
#from google.colab import drive
#drive.mount("/content/drive")

In [3]:
# Set wd
os.chdir('/Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas')

print("Working directory:", os.getcwd())

Working directory: /Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas


# Import csv of correctly/wrongly labelled service stations and extract associated polygons

In [4]:
# Load correctly_labelled_service_stations csv and transform to list
correctly_labelled_service_stations_path = "02_data_acquisition/verified_parking_data/service_stations/correctly_labelled_service_stations.csv"
wrongly_labelled_service_stations_path = "02_data_acquisition/verified_parking_data/service_stations/wrongly_labelled_service_stations.csv"

# Read the CSV file into a pandas DataFrame
correctly_labelled_service_stations = pd.read_csv(correctly_labelled_service_stations_path)
wrongly_labelled_service_stations = pd.read_csv(wrongly_labelled_service_stations_path)

# Transform df to list
correctly_labelled_service_stations_list = []
for row in correctly_labelled_service_stations.values.tolist():
    correctly_labelled_service_stations_list.extend(row)

wrongly_labelled_service_stations_list = []
for row in wrongly_labelled_service_stations.values.tolist():
    wrongly_labelled_service_stations_list.extend(row)

# Print first 5 items of correctly_labelled_service_stations_list
print(correctly_labelled_service_stations[:5])
print("\n")
print("Number of correctly labelled service stations:", len(correctly_labelled_service_stations_list))

                         id_rest
0   lon_8.6630029_lat_50.2542348
1  lon_11.3531491_lat_50.9378362
2   lon_10.2712414_lat_53.331074
3   lon_9.6300119_lat_53.8303266
4   lon_9.6297338_lat_53.8301666


Number of correctly labelled service stations: 296


In [5]:
# Load the GeoJSON files containing service station polygons and truck/car/quick parking polygons together with service station IDs

corrected_parking = gpd.read_file("02_data_acquisition/verified_parking_data/parking/corrected_parking.geojson") # polygons of car, truck and quick parking space
rest_stations = gpd.read_file("02_data_acquisition/parking_data/rest_stations.geojson") # polygons and ids of rest stations

In [6]:
rest_stations.shape

(506, 5)

In [7]:
# Create geopandas geodataframes that only contain polygons of correctly/wrongly labelled service stations

print("Shape of corrected_parking before dropping irrelevant rows:", corrected_parking.shape)
print("Shape of rest_stations before dropping irrelevant rows:", rest_stations.shape)

# Drop all rows not corresponding to correctly labelled service stations
parking_at_correct= corrected_parking[corrected_parking["id_rest"].isin(correctly_labelled_service_stations_list)]
correctly_labelled_service_stations = rest_stations[rest_stations["id"].isin(correctly_labelled_service_stations_list)]

# Drop all rows corresponding to correctly labelled service stations
parking_at_wrong = corrected_parking[corrected_parking["id_rest"].isin(wrongly_labelled_service_stations_list)]
wrongly_labelled_service_stations = rest_stations[rest_stations["id"].isin(wrongly_labelled_service_stations_list)]

print("Shape of parking_at_correct:", parking_at_correct.shape)
print("Shape of correctly_labelled_service_stations:", correctly_labelled_service_stations.shape)

print("Shape of parking_at_wrong:", parking_at_wrong.shape)
print("Shape of wrongly_labelled_service_stations:", wrongly_labelled_service_stations.shape)

Shape of corrected_parking before dropping irrelevant rows: (1606, 6)
Shape of rest_stations before dropping irrelevant rows: (506, 5)
Shape of parking_at_correct: (1304, 6)
Shape of correctly_labelled_service_stations: (296, 5)
Shape of parking_at_wrong: (299, 6)
Shape of wrongly_labelled_service_stations: (208, 5)


## Clean up dataframes

In [8]:
# Drop irrelevant columns, reorder columns, reset index of parking_at_correct

parking_at_correct = parking_at_correct.drop(["id_car", "name"], axis = 1)
parking_at_correct = parking_at_correct.rename(columns={"@id": "id_OSM_parking"})
parking_at_correct = parking_at_correct.reindex(columns=["id_rest", "id_OSM_parking", "type", "geometry"])
parking_at_correct = parking_at_correct.reset_index(drop = True)

parking_at_correct.head()

,id_rest,id_OSM_parking,type,geometry
0,lon_10.2712414_lat_53.331074,relation/13277623,car,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53..."
1,lon_7.5846163_lat_53.2588511,way/33526884,car,"POLYGON ((7.58527 53.25883, 7.58618 53.25869, ..."
2,lon_7.2005154_lat_52.2816225,way/38125314,car,"POLYGON ((7.19685 52.28128, 7.19692 52.28127, ..."
3,lon_10.7306355_lat_50.559048,way/48905621,car,"POLYGON ((10.73083 50.55915, 10.73087 50.55881..."
4,lon_10.7298729_lat_50.5605634,way/48925325,car,"POLYGON ((10.72986 50.56033, 10.72980 50.56026..."


In [9]:
# Drop irrelevant columns, reorder columns, reset index of correctly_labelled_service_stations

correctly_labelled_service_stations = correctly_labelled_service_stations.rename(columns={"id": "id_rest", "@id": "id_OSM_rest"})
correctly_labelled_service_stations = correctly_labelled_service_stations.drop("highway", axis = 1)
correctly_labelled_service_stations = correctly_labelled_service_stations.reset_index(drop = True)

correctly_labelled_service_stations.head()

,id_rest,id_OSM_rest,name,geometry
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"POLYGON ((11.35315 50.93784, 11.35315 50.93783..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"POLYGON ((10.27124 53.33107, 10.27096 53.33121..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"POLYGON ((9.63001 53.83033, 9.63273 53.82905, ..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"POLYGON ((9.62973 53.83017, 9.62965 53.83010, ..."


In [10]:
# Drop irrelevant columns, reorder columns, reset index of parking_at_wrong

parking_at_wrong = parking_at_wrong.drop(["id_car", "name"], axis = 1)
parking_at_wrong = parking_at_wrong.rename(columns={"@id": "id_OSM_parking"})
parking_at_wrong = parking_at_wrong.reindex(columns=["id_rest", "id_OSM_parking", "type", "geometry"])
parking_at_wrong = parking_at_wrong.reset_index(drop = True)

parking_at_wrong.head()

,id_rest,id_OSM_parking,type,geometry
0,lon_10.2703215_lat_53.3301407,relation/13277622,car,"MULTIPOLYGON (((10.26903 53.33039, 10.26926 53..."
1,lon_12.140788_lat_52.2431929,relation/15482166,truck,"MULTIPOLYGON (((12.14216 52.24271, 12.14213 52..."
2,lon_12.149456_lat_52.2449316,relation/15482306,car,"MULTIPOLYGON (((12.14713 52.24497, 12.14717 52..."
3,lon_9.323161_lat_54.0686721,way/31129920,car,"POLYGON ((9.32127 54.06898, 9.32314 54.06861, ..."
4,lon_11.2481831_lat_52.1991827,way/42750034,car,"POLYGON ((11.24512 52.19985, 11.24526 52.19971..."


In [11]:
# Drop irrelevant columns, reorder columns, reset index of wrongly_labelled_service_stations

wrongly_labelled_service_stations = wrongly_labelled_service_stations.rename(columns={"id": "id_rest", "@id": "id_OSM_rest"})
wrongly_labelled_service_stations = wrongly_labelled_service_stations.drop("highway", axis = 1)
wrongly_labelled_service_stations = wrongly_labelled_service_stations.reset_index(drop = True)

wrongly_labelled_service_stations.head()

,id_rest,id_OSM_rest,name,geometry
0,lon_11.2442867_lat_51.3248863,relation/3947321,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,Probstheide,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,relation/4672228,Ladeburger Heide,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6624068_lat_50.2544468,way/22568868,Spießwald,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."
4,lon_8.6184146_lat_49.7896285,way/28329065,Rastplatz Rolandshöhe,"POLYGON ((8.61841 49.78963, 8.61857 49.79067, ..."


## Download dataframes

In [12]:
# Check shapes of dataframes

print("Shape of parking_at_correct:", parking_at_correct.shape)
print("Shape of correctly_labelled_service_stations:", correctly_labelled_service_stations.shape)
print("Shape of parking_at_wrong:", parking_at_wrong.shape)
print("Shape of wrongly_labelled_service_stations:", wrongly_labelled_service_stations.shape)

Shape of parking_at_correct: (1304, 4)
Shape of correctly_labelled_service_stations: (296, 4)
Shape of parking_at_wrong: (299, 4)
Shape of wrongly_labelled_service_stations: (208, 4)


In [211]:
# Download geopandas geodataframes as GeoJSONs

# Specify the path where you want to save the GeoJSON file
output_path_parking_at_correct = "02_data_acquisition/verified_parking_data/parking/parking_at_correct.geojson"
output_path_correctly_labelled_service_stations = "02_data_acquisition/verified_parking_data/service_stations/correctly_labelled_service_stations.geojson"
output_path_parking_at_wrong = "02_data_acquisition/verified_parking_data/parking/parking_at_wrong.geojson"
output_path_wrongly_labelled_service_stations = "02_data_acquisition/verified_parking_data/service_stations/wrongly_labelled_service_stations.geojson"

# Export the GeoPandas DataFrame to GeoJSON
parking_at_correct.to_file(output_path_parking_at_correct, driver='GeoJSON')
correctly_labelled_service_stations.to_file(output_path_correctly_labelled_service_stations, driver='GeoJSON')
parking_at_wrong.to_file(output_path_parking_at_wrong, driver='GeoJSON')
wrongly_labelled_service_stations.to_file(output_path_wrongly_labelled_service_stations, driver='GeoJSON')

# Load GeoJSONs and define bounding boxes for download

In [13]:
# Load the GeoJSON files of correctly labelled service stations and parking areas
parking_at_correct = gpd.read_file("02_data_acquisition/verified_parking_data/parking/parking_at_correct.geojson") # polygons of parkings areas at correctly labelled service stations
correctly_labelled_service_stations = gpd.read_file("02_data_acquisition/verified_parking_data/service_stations/correctly_labelled_service_stations.geojson") # polygons of verified rest stops

# Load the GeoJSON files of wrongly labelled service stations
parking_at_wrong = gpd.read_file("02_data_acquisition/verified_parking_data/parking/parking_at_wrong.geojson") # polygons of parkings areas at wrongly labelled service stations

In [14]:
parking_at_correct

,id_rest,id_OSM_parking,type,geometry
0,lon_10.2712414_lat_53.331074,relation/13277623,car,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53..."
1,lon_7.5846163_lat_53.2588511,way/33526884,car,"POLYGON ((7.58527 53.25883, 7.58618 53.25869, ..."
2,lon_7.2005154_lat_52.2816225,way/38125314,car,"POLYGON ((7.19685 52.28128, 7.19692 52.28127, ..."
3,lon_10.7306355_lat_50.559048,way/48905621,car,"POLYGON ((10.73083 50.55915, 10.73087 50.55881..."
4,lon_10.7298729_lat_50.5605634,way/48925325,car,"POLYGON ((10.72986 50.56033, 10.72980 50.56026..."
...,...,...,...,...
1299,lon_11.0467665_lat_51.4688222,way/1194289084,truck,"POLYGON ((11.04380 51.46788, 11.04414 51.46784..."
1300,lon_11.0467665_lat_51.4688222,way/1194289085,truck,"POLYGON ((11.04439 51.46815, 11.04445 51.46815..."
1301,lon_11.0467665_lat_51.4688222,way/1194289086,truck,"POLYGON ((11.04503 51.46846, 11.04506 51.46846..."
1302,lon_11.0467665_lat_51.4688222,way/1194289087,quick,"POLYGON ((11.04375 51.46758, 11.04399 51.46763..."


In [15]:
correctly_labelled_service_stations

,id_rest,id_OSM_rest,name,geometry
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"POLYGON ((11.35315 50.93784, 11.35315 50.93783..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"POLYGON ((10.27124 53.33107, 10.27096 53.33121..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"POLYGON ((9.63001 53.83033, 9.63273 53.82905, ..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"POLYGON ((9.62973 53.83017, 9.62965 53.83010, ..."
...,...,...,...,...
291,lon_12.7580653_lat_52.8778128,way/915630046,Am Rhinluch,"POLYGON ((12.75807 52.87781, 12.75786 52.87778..."
292,lon_7.2005154_lat_52.2816225,way/988014767,Gut Friedrichstal,"POLYGON ((7.20052 52.28162, 7.20053 52.28167, ..."
293,lon_8.5597202_lat_49.5969765,way/992004892,Wildbahn,"POLYGON ((8.55972 49.59698, 8.55958 49.59693, ..."
294,lon_10.7306355_lat_50.559048,way/1171916133,Adlersberg,"POLYGON ((10.73064 50.55905, 10.73067 50.55851..."


In [16]:
wrongly_labelled_service_stations

,id_rest,id_OSM_rest,name,geometry
0,lon_11.2442867_lat_51.3248863,relation/3947321,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,Probstheide,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,relation/4672228,Ladeburger Heide,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6624068_lat_50.2544468,way/22568868,Spießwald,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."
4,lon_8.6184146_lat_49.7896285,way/28329065,Rastplatz Rolandshöhe,"POLYGON ((8.61841 49.78963, 8.61857 49.79067, ..."
...,...,...,...,...
203,lon_7.2459984_lat_52.3914582,way/1024211059,Rastplatz Bergler Feld Ost,"POLYGON ((7.24600 52.39146, 7.24561 52.38924, ..."
204,lon_7.2465958_lat_52.3950579,way/1024211060,Rastplatz Bergler Feld West,"POLYGON ((7.24660 52.39506, 7.24656 52.39511, ..."
205,lon_8.5358385_lat_49.5561227,way/1075944398,Rennschlag,"POLYGON ((8.53584 49.55612, 8.53523 49.55625, ..."
206,lon_8.4350744_lat_50.607106,way/1104993376,Park und Rastplatz Am Behlkopf,"POLYGON ((8.43507 50.60711, 8.43507 50.60706, ..."


## Determine download bounding boxes


### Define function

In [17]:
def create_image_bounding_box(polygon, bbox_size_m = 500):
    """
    Create an output bounding box that fully covers the input bounding box and is of size bbox_size_m x bbox_size_m.

    Parameters:
    polygon (shapely Polygon): input polygon
    bbox_size_m (int): size of the output bounding box

    Returns:
    The output bounding box as [left, bottom, right, top]
    """

    minx, miny, maxx, maxy = polygon.bounds

    # Calculate the centre of the input bounding box
    centre_x = (miny + maxy) / 2
    centre_y = (minx + maxx) / 2

    distance_from_centre = bbox_size_m / 2

    # Calculate the coordinates distance_from_centre meters north, south, east, and west of the center
    north = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=0).latitude
    south = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=180).latitude
    east = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=90).longitude
    west = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=270).longitude

    # Calculate the maximum allowable shift in each direction to keep the input bbox within the output bbox
    max_shift_north = north - maxy
    max_shift_south = miny - south
    max_shift_east = east - maxx
    max_shift_west = minx - west

    # Randomly shift the center within the allowable range
    shift_x = random.uniform(-min(max_shift_west, distance_from_centre), min(max_shift_east, distance_from_centre))
    shift_y = random.uniform(-min(max_shift_south, distance_from_centre), min(max_shift_north, distance_from_centre))

    # Calculate the final output bounding box
    output_bbox = [
        west + shift_x,  # left
        south + shift_y, # bottom
        east + shift_x,  # right
        north + shift_y  # top
    ]

    return output_bbox

In [18]:
def determine_bbox_max_span_m(bbox):
    """
    Determine the size of a bounding box in meters.

    Parameters:
    bbox (list): bounding box as [left, bottom, right, top]
    """

    minx, miny, maxx, maxy = bbox

    x_line = shapely.LineString([(minx, miny), (maxx, miny)])
    y_line = shapely.LineString([(minx, miny), (minx, maxy)])

    geod = Geod(ellps="WGS84")

    length_x = math.ceil(geod.geometry_length(x_line))
    length_y = math.ceil(geod.geometry_length(y_line))

    print(f"The bounding box has a width of: {length_x}m and a height of: {length_y}m")

### Test function

In [143]:
# Extract random rest stop polygon

random_index = random.randint(0, 266)
print("Random index:", random_index)

# Extract polygon
test_polygon = correctly_labelled_service_stations.iloc[random_index]["geometry"]
print(test_polygon)

Random index: 242
POLYGON ((8.0316758 52.415733, 8.0315558 52.415725, 8.032054 52.4121665, 8.0321693 52.4121753, 8.032627 52.4122906, 8.0330226 52.4127078, 8.033087 52.4129957, 8.0328295 52.4145074, 8.032328 52.4155979, 8.0318828 52.4158139, 8.0316758 52.415733))


In [144]:
# Create bounding box
covering_test_bbox = create_image_bounding_box(test_polygon)
covering_test_bbox

[8.029274592923368, 52.41153767053867, 8.036622921856104, 52.416031034526846]

In [145]:
# Check bounding box size in meter
determine_bbox_max_span_m(covering_test_bbox)

The bounding box has a width of: 501m and a height of: 500m


A bounding box of around 500mx500m ensures the pixel resolution of at least 0.2m/pixel when downloading 2560x2560 pixel images.

### Calculate bbox for every rest stop

In [19]:
# Set random seed
random.seed(103)

# Create image bounding boxes for every rest station
correctly_labelled_service_stations["bbox"] = correctly_labelled_service_stations['geometry'].apply(create_image_bounding_box)
wrongly_labelled_service_stations["bbox"] = wrongly_labelled_service_stations['geometry'].apply(create_image_bounding_box)

In [20]:
# Change order of columns to comply with geopandas convention

correctly_labelled_service_stations = correctly_labelled_service_stations[["id_rest", "id_OSM_rest", "name", "bbox", "geometry"]]
wrongly_labelled_service_stations = wrongly_labelled_service_stations[["id_rest", "id_OSM_rest", "name", "bbox", "geometry"]]

In [21]:
correctly_labelled_service_stations

,id_rest,id_OSM_rest,name,bbox,geometry
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"[8.662897964530508, 50.24947031826926, 8.66990...","POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"[11.352126356256656, 50.93673132065365, 11.359...","POLYGON ((11.35315 50.93784, 11.35315 50.93783..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"[10.264345359109937, 53.328233742563924, 10.27...","POLYGON ((10.27124 53.33107, 10.27096 53.33121..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"[9.629200346595411, 53.82616873812249, 9.63679...","POLYGON ((9.63001 53.83033, 9.63273 53.82905, ..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"[9.628319773449437, 53.82730601524238, 9.63591...","POLYGON ((9.62973 53.83017, 9.62965 53.83010, ..."
...,...,...,...,...,...
291,lon_12.7580653_lat_52.8778128,way/915630046,Am Rhinluch,"[12.754311642753441, 52.87373430027324, 12.761...","POLYGON ((12.75807 52.87781, 12.75786 52.87778..."
292,lon_7.2005154_lat_52.2816225,way/988014767,Gut Friedrichstal,"[7.196198912998569, 52.27916482977726, 7.20352...","POLYGON ((7.20052 52.28162, 7.20053 52.28167, ..."
293,lon_8.5597202_lat_49.5969765,way/992004892,Wildbahn,"[8.554638173536537, 49.59453943341597, 8.56155...","POLYGON ((8.55972 49.59698, 8.55958 49.59693, ..."
294,lon_10.7306355_lat_50.559048,way/1171916133,Adlersberg,"[10.724665021750997, 50.55768307908295, 10.731...","POLYGON ((10.73064 50.55905, 10.73067 50.55851..."


In [22]:
wrongly_labelled_service_stations

,id_rest,id_OSM_rest,name,bbox,geometry
0,lon_11.2442867_lat_51.3248863,relation/3947321,Hohe Schrecke West,"[11.241064915797857, 51.32243637400286, 11.248...","MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,Probstheide,"[13.540777948956048, 52.71013054268637, 13.548...","POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,relation/4672228,Ladeburger Heide,"[13.5452955790032, 52.70705615941434, 13.55269...","POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6624068_lat_50.2544468,way/22568868,Spießwald,"[8.660065778552887, 50.25160742899144, 8.66707...","POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."
4,lon_8.6184146_lat_49.7896285,way/28329065,Rastplatz Rolandshöhe,"[8.616622212048906, 49.788430865137876, 8.6235...","POLYGON ((8.61841 49.78963, 8.61857 49.79067, ..."
...,...,...,...,...,...
203,lon_7.2459984_lat_52.3914582,way/1024211059,Rastplatz Bergler Feld Ost,"[7.24460460097266, 52.38743780607369, 7.251949...","POLYGON ((7.24600 52.39146, 7.24561 52.38924, ..."
204,lon_7.2465958_lat_52.3950579,way/1024211060,Rastplatz Bergler Feld West,"[7.243070829595409, 52.39210190678455, 7.25041...","POLYGON ((7.24660 52.39506, 7.24656 52.39511, ..."
205,lon_8.5358385_lat_49.5561227,way/1075944398,Rennschlag,"[8.52982204213126, 49.55406843116357, 8.536732...","POLYGON ((8.53584 49.55612, 8.53523 49.55625, ..."
206,lon_8.4350744_lat_50.607106,way/1104993376,Park und Rastplatz Am Behlkopf,"[8.431103744122044, 50.60606866341377, 8.43816...","POLYGON ((8.43507 50.60711, 8.43507 50.60706, ..."


In [24]:
# Delete geometry column from correctly_labelled_service_stations
correctly_labelled_service_stations_bbox = correctly_labelled_service_stations.drop("geometry", axis = 1)

# Delete geometry column from wrongly_labelled_service_stations
wrongly_labelled_service_stations_bbox = wrongly_labelled_service_stations.drop("geometry", axis = 1)

# Download correctly_labelled_service_stations as csv
output_path_rest_stations_ver_bbox = "02_data_acquisition/verified_parking_data/bounding_boxes/correctly_labelled_service_stations_bbox.csv"
correctly_labelled_service_stations_bbox.to_csv(output_path_rest_stations_ver_bbox, index = False)

# Download wrongly_labelled_service_stations as csv
output_path_rest_stations_wrong_bbox = "02_data_acquisition/verified_parking_data/bounding_boxes/wrongly_labelled_service_stations_bbox.csv"
wrongly_labelled_service_stations_bbox.to_csv(output_path_rest_stations_wrong_bbox, index = False)

In [25]:
correctly_labelled_service_stations_bbox

,id_rest,id_OSM_rest,name,bbox
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"[8.662897964530508, 50.24947031826926, 8.66990..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"[11.352126356256656, 50.93673132065365, 11.359..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"[10.264345359109937, 53.328233742563924, 10.27..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"[9.629200346595411, 53.82616873812249, 9.63679..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"[9.628319773449437, 53.82730601524238, 9.63591..."
...,...,...,...,...
291,lon_12.7580653_lat_52.8778128,way/915630046,Am Rhinluch,"[12.754311642753441, 52.87373430027324, 12.761..."
292,lon_7.2005154_lat_52.2816225,way/988014767,Gut Friedrichstal,"[7.196198912998569, 52.27916482977726, 7.20352..."
293,lon_8.5597202_lat_49.5969765,way/992004892,Wildbahn,"[8.554638173536537, 49.59453943341597, 8.56155..."
294,lon_10.7306355_lat_50.559048,way/1171916133,Adlersberg,"[10.724665021750997, 50.55768307908295, 10.731..."


In [26]:
wrongly_labelled_service_stations_bbox

,id_rest,id_OSM_rest,name,bbox
0,lon_11.2442867_lat_51.3248863,relation/3947321,Hohe Schrecke West,"[11.241064915797857, 51.32243637400286, 11.248..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,Probstheide,"[13.540777948956048, 52.71013054268637, 13.548..."
2,lon_13.549042_lat_52.7092396,relation/4672228,Ladeburger Heide,"[13.5452955790032, 52.70705615941434, 13.55269..."
3,lon_8.6624068_lat_50.2544468,way/22568868,Spießwald,"[8.660065778552887, 50.25160742899144, 8.66707..."
4,lon_8.6184146_lat_49.7896285,way/28329065,Rastplatz Rolandshöhe,"[8.616622212048906, 49.788430865137876, 8.6235..."
...,...,...,...,...
203,lon_7.2459984_lat_52.3914582,way/1024211059,Rastplatz Bergler Feld Ost,"[7.24460460097266, 52.38743780607369, 7.251949..."
204,lon_7.2465958_lat_52.3950579,way/1024211060,Rastplatz Bergler Feld West,"[7.243070829595409, 52.39210190678455, 7.25041..."
205,lon_8.5358385_lat_49.5561227,way/1075944398,Rennschlag,"[8.52982204213126, 49.55406843116357, 8.536732..."
206,lon_8.4350744_lat_50.607106,way/1104993376,Park und Rastplatz Am Behlkopf,"[8.431103744122044, 50.60606866341377, 8.43816..."


# Set up tile services

## WMS and layer information

This code was partly taken from: [OWSLib 0.29.3 documentation -> Examples -> Interact with a WMS](https://owslib.readthedocs.io/en/stable/notebooks/wms.html)

In [152]:
def get_wms_information(wms_url):
    """
    Output all necessary WMS information for the WMS service provided.

    Parameters:
    wms_url: The GetCapabilities url of the WMS service

    Returns:
    A dictionary with version, url, title, abstract, provider_name, layer_names, operations, operations_urls, format_options
    """

    wms = WebMapService(wms_url, version="1.3.0")

    version = wms.identification.version
    title = wms.identification.title
    abstract = wms.identification.abstract
    provider_name = wms.provider.name
    layer_names = list(wms.contents)
    operations = [op.name for op in wms.operations]
    operations_urls = wms.getOperationByName("GetMap").methods
    format_options = wms.getOperationByName("GetMap").formatOptions

    wms_information = {"version": version,
                       "url": wms_url,
                       "title": title,
                       "abstract": abstract,
                       "provider_name": provider_name,
                       "layer_names": layer_names,
                       "operations": operations,
                       "operations_urls": operations_urls,
                       "format_options": format_options
                       }

    return wms_information

def get_layer_information(wms_url, layer_name):
    """
    Retrieve information about extent and crs_options of selected layer.

    Paramters:
    wms_url: The GetCapabilities url of the WMS service
    layer_name: The name of the selected layer

    Returns:
    A dictionary with layer_name, extent, crs_options.
    """

    wms = WebMapService(wms_url, version="1.3.0")

    extent = wms.contents[layer_name].boundingBoxWGS84
    crs_options = wms[layer_name].crsOptions

    layer_information = {"layer_name": layer_name,
                         "extent": extent,
                         "crs_options": crs_options
                         }

    return layer_information

In [153]:
# Define WMS service and layer to be tested

test_wms_url = "https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS"
test_layer = "WMS_NI_DOP"

In [154]:
# Return all relevant WMS service information
get_wms_information(test_wms_url)

{'version': '1.3.0',
 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS',
 'title': 'WMS NI DOP',
 'abstract': 'Digitale Orthophotos Niedersachsen, Bodenauflösung 20 cm (DOP20)',
 'provider_name': 'Landesamt für Geoinformation und Landesvermessung Niedersachsen (LGLN) - Landesbetrieb Landesvermessung und Geobasisinformation',
 'layer_names': ['WMS_NI_DOP', 'dop20'],
 'operations': ['GetCapabilities', 'GetMap', 'GetFeatureInfo'],
 'operations_urls': [{'type': 'Get',
   'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'},
  {'type': 'Post',
   'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'}],
 'format_options': ['image/png',
  'image/jpeg',
  'image/png; mode=8bit',
  'image/vnd.jpeg-png',
  'image/vnd.jpeg-png8',
  'application/x-pdf',
  'image/svg+xml',
  'image/tiff',
  'application/vnd.google-earth.kml+xml',
  'application/vnd.google-earth.kmz',
  'application/vn

In [155]:
# Return all relevant layer information
get_layer_information(test_wms_url, test_layer)

{'layer_name': 'WMS_NI_DOP',
 'extent': (6.505772, 51.153098, 11.754046, 54.148101),
 'crs_options': ['EPSG:4647',
  'EPSG:3857',
  'EPSG:3035',
  'EPSG:4258',
  'EPSG:31469',
  'EPSG:3034',
  'EPSG:32632',
  'EPSG:31466',
  'EPSG:25833',
  'EPSG:31467',
  'EPSG:4326',
  'EPSG:31468',
  'EPSG:3044',
  'EPSG:25832',
  'EPSG:3045',
  'EPSG:32633']}

## Return image from WMS bbox

In [156]:
# Define WMS service and layer for each German state

wms_urls = {
        'WMS_NI_DOP': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS', # Lower Saxony
        'bebb_dop20c': 'https://isk.geobasis-bb.de/mapproxy/dop20c/service/wms?Request=GetCapabilities&Service=WMS', # Brandenburg & Berlin
        'th_dop': 'https://www.geoproxy.geoportal-th.de/geoproxy/services/DOP?REQUEST=GetCapabilities&version=1.1.1&service=WMS', # Thuringia
        'lsa_lvermgeo_dop20_2': 'https://www.geodatenportal.sachsen-anhalt.de/wss/service/ST_LVermGeo_DOP_WMS_OpenData/guest?', # Saxony-Anhalt
        'he_dop_rgb': 'https://gds-srv.hessen.de/cgi-bin/lika-services/ogc-free-images.ows?', # Hessia
        'sh_dop20_rgb': 'https://dienste.gdi-sh.de/WMS_SH_DOP20col_OpenGBD?Service=wms&version=1.3.0&request=getCapabilities', # Schleswig-Holstein
    }

*   sh_dop20_rgb has an overly large, inaccurate extent. For this reason, this layer is tested last

In [157]:
# Print WMS and layer information for all selected Länder layers
counter = 1

for layer, url in wms_urls.items():
    print(f"wms_{counter}", get_wms_information(url))
    print(f"layer_{counter}", get_layer_information(url, layer))
    print("\n")
    counter += 1

wms_1 {'version': '1.3.0', 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS', 'title': 'WMS NI DOP', 'abstract': 'Digitale Orthophotos Niedersachsen, Bodenauflösung 20 cm (DOP20)', 'provider_name': 'Landesamt für Geoinformation und Landesvermessung Niedersachsen (LGLN) - Landesbetrieb Landesvermessung und Geobasisinformation', 'layer_names': ['WMS_NI_DOP', 'dop20'], 'operations': ['GetCapabilities', 'GetMap', 'GetFeatureInfo'], 'operations_urls': [{'type': 'Get', 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'}, {'type': 'Post', 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'}], 'format_options': ['image/png', 'image/jpeg', 'image/png; mode=8bit', 'image/vnd.jpeg-png', 'image/vnd.jpeg-png8', 'application/x-pdf', 'image/svg+xml', 'image/tiff', 'application/vnd.google-earth.kml+xml', 'application/vnd.google-earth.kmz', 'application/vnd.mapbox-vector-tile', 'applic

**Findings:**

*   All layers have the option to select the EPSG:4326 coordinate system, a coordinate system on the WGS84 reference ellipsoid
*   WGS84 is also used as the standard coordinates system used by GSP and also OSM
*   Only Niedersachsen, Thüringen and Hessen can export in Tiff format

In [158]:
def retrieve_layer_extents(wms_urls):
        """
        Retrieve layer extents of the WMS urls.

        Parameters:
        wms_urls (dict): dictionary of layer names and WMS GetCapability URLs

        Returns:
        A dictionary of the extents of the WMS layers.
        """
        wms_layer_extents = {}

        for layer_name, url in wms_urls.items():
            bounding_box = get_layer_information(url, layer_name)["extent"]
            wms_layer_extents[layer_name] = bounding_box

        return wms_layer_extents

In [159]:
# Extract layer extents
layer_extents = retrieve_layer_extents(wms_urls)
print(layer_extents)

{'WMS_NI_DOP': (6.505772, 51.153098, 11.754046, 54.148101), 'bebb_dop20c': (11.152768795679583, 51.2635170116316, 15.009068839315324, 53.61004915755329), 'th_dop': (9.70043908, 50.13516795, 12.75958006, 51.7188448), 'lsa_lvermgeo_dop20_2': (10.5092, 50.8927, 13.3233, 53.0769), 'he_dop_rgb': (7.41867, 49.25, 10.5, 51.7596), 'sh_dop20_rgb': (0.105946742406, 45.237542736, 20.4488912945, 56.8478734515)}


In [160]:
def create_png_from_bbox(bbox, wms_urls, layer_extents, size = 2560):
    """
    Create square png map from a bounding box sourced from WMS services.

    Parameters:
    bbox (tuple): (left, bottom, right, top)
    wms_urls (dict): dictionary containing layer_names as keys and urls as values
    layer_extents (dict): dictionary containing layer_names as keys and layer extent tuples as values
    size (int): determines the size of the output png in pixels

    Returns:
    A square png image of the map.
    """


    def bbox_intersects(bbox1, bbox2):
        """
        Check if two bounding boxes intersect.

        Parameters:
        bbox1, bbox2 (tuple): tuple of bbox corners (left, bottom, right, top)

        Returns:
        True if bboxes intersect, False otherwise
        """
        return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or
                    bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])


    def is_blank_image(image):
        """
        Check whether an image is blank or not.

        Parameters:
        image (image object): image of WMS service

        Returns:
        True if image is blank, False otherwise.
        """
        try:
            # Open the image from the image object
            image = Image.open(io.BytesIO(image.read()))

            # Convert the image to a numpy array
            image_array = np.array(image)

            # Depending on the image mode, check whether image is completely white or black
            if image.mode == 'RGB':
                # Check if the image is completely white or black
                is_white = np.all(image_array == [255, 255, 255], axis=(-1)).all()
                is_black = np.all(image_array == [0, 0, 0], axis=(-1)).all()
            elif image.mode == 'RGBA':
                # Check if the image is completely white or black
                is_white = np.all(image_array[:, :, :3] == [255, 255, 255], axis=(-1)).all()
                is_black = np.all(image_array[:, :, :3] == [0, 0, 0], axis=(-1)).all()
            else:
                # Raise error for other image mode
                raise ValueError("Unsupported image mode.")

            return is_white or is_black

        except Exception as e:
            print(f"An error occurred: {e}")
            return False


    def determine_layer_for_bbox(bbox, wms_urls, layer_extents):
        """
        Determine which layer to use based for the provided bounding box.

        Parameters:
        bbox (tuple): bounding box (left, bottom, right, top)
        wms_urls (dict): dictionary containing layer_names as keys and urls as values
        layer_extents: dictionary of layer names and their extent bounding boxes

        Returns:
        Layer name or None if no intersection
        """
        # Check WMSs one after the other until image is returned
        for layer, extent in layer_extents.items():
            wms = WebMapService(wms_urls[layer])
            response = wms.getmap(layers=[layer],
                              srs='EPSG:4326',
                              bbox=bbox,
                              size=(2, 2),
                              format='image/png')
            # First, check if bounding box intersects
            if not bbox_intersects(bbox, extent):
                print(f"{layer} does not contain bounding box.")
            # Second, check whether image is blank
            elif is_blank_image(response):
                print(f"{layer} returns blank image.")
            # Return image
            else:
                print(f"{layer} returns image.")

                return layer
        return None

    # Determine the layer that intersects the bounding box
    chosen_layer = determine_layer_for_bbox(bbox, wms_urls, layer_extents)

    if chosen_layer == None:
        raise ValueError("No layer intersects with the provided bounding box.")
    else:
        # Correctly create a WebMapService object from chosen layer
        wms = WebMapService(wms_urls[chosen_layer])
        response = wms.getmap(layers=[chosen_layer],
                              srs='EPSG:4326',
                              bbox=bbox,
                              size=(size, size),
                              format='image/png')
    # Return the image
    return response

In [161]:
def pixel_dimensions(image):
    """
    Return pixel dimensions of an image.

    Parameters:
    image (PIL Image object)

    Returns:
    A tuple of width and height dimensions of the input image.
    """

    image_data = BytesIO(image.read())
    image_loaded = Image.open(image_data)

    # Get image dimensions (Width x Height)
    width, height = image_loaded.size

    return (width, height)

# Download images

## Define downloading functions

In [162]:
def download_as_png(image, filepath):
    """
    Download an image as a PNG file.

    Parameters:
    image (PIL Image object)
    filepath (str): path to save the image
    """
    # Assuming img_new is your image response object
    image_data = BytesIO(image.read())

    # Write the image data to a file
    with open(filepath, 'wb') as file:
        file.write(image_data.getvalue())

In [163]:
def download_tif_from_png(png_path, tif_path, bbox):
    """
    Convert a PNG image to a GeoTIFF file.

    Parameters:
    png_path (str): path to the PNG image
    tif_path (str): path to save the GeoTIFF file
    bbox (list): bounding box as [left, bottom, right, top]
    """
    # Load the PNG image as a Rasterio dataset
    png = rasterio.open(png_path)

    # Read the data from the dataset into a NumPy array
    image_array = png.read()

    # Define transformation from bounding box
    left, bottom, right, top = bbox
    transform = from_bounds(left, bottom, right, top, png.width, png.height)

    # Update metadata
    meta = {
        'driver': 'GTiff',
        'dtype': png.dtypes[0],
        'nodata': None,
        'width': png.width,
        'height': png.height,
        'count': png.count,
        'crs': CRS.from_epsg(4326),  # WGS84
        'transform': transform
    }

    # Write the array to a new TIFF file
    with rasterio.open(tif_path, 'w', **meta) as dst:
        dst.write(image_array)

In [164]:
def delete_file(file_path):
    """
    Delete a file.

    Parameters:
    file_path (str): path to the file to be deleted
    """
    # Check if the file exists
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File '{file_path}' does not exist.")

## Test Download

In [ ]:
# Define test index
test_index = 4

test_bbox = correctly_labelled_service_stations.iloc[test_index].loc["bbox"]
test_id_rest = correctly_labelled_service_stations.iloc[test_index].loc["id_rest"]
test_name = correctly_labelled_service_stations.iloc[test_index].loc["name"]

print("Test bounding box:", test_bbox)
print("Test id box:", test_id_rest)
print("Test service station:", test_name)

In [ ]:
# Create map for test service station
response = create_png_from_bbox(test_bbox, wms_urls, layer_extents, size = 2560)

In [ ]:
# Display service station
displayImage(response.read())

In [ ]:
# Print type and pixel dimensions
print(type(response))
print("Image pixel dimensions", pixel_dimensions(response))

In [ ]:
# Download image as PNG
file_path_png = f"02_data_acquisition/test_image_download/{test_id_rest}_{test_name}.png"
download_as_png(response, file_path_png)

In [ ]:
# Download image as tif from png
file_path_tif = f"02_data_acquisition/test_image_download/{test_id_rest}_{test_name}.tif"
download_tif_from_png(file_path_png, file_path_tif, test_bbox)

In [ ]:
# Delete PNG function

def delete_file(file_path):
    # Check if the file exists
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File '{file_path}' has been deleted.")
    else:
        print(f"File '{file_path}' does not exist.")

In [ ]:
# Delete Png
file_path_png = file_path_png
delete_file(file_path_png)

## Mass-Download correctly laballed service stations as tifs

In [165]:
def mass_download_tif(geodataframe, wms_urls, layer_extents, size = 2560):
    """
    Download a large number of GeoTIFF images from WMS services

    Parameters:
    geodataframe (GeoDataFrame): GeoDataFrame containing the bounding boxes
    wms_urls (dict): dictionary containing layer_names as keys and urls as values
    layer_extents: dictionary of layer names and their extent bounding boxes
    size (int): determines the size of the output png in pixels
    """

    for index, row in geodataframe.iterrows():
        bbox = row["bbox"]
        id_rest = row["id_rest"]
        name = row["name"]

        print(f"Downloading image {index}: {name} ({id_rest})\n")

        # Create map
        response = create_png_from_bbox(bbox, wms_urls, layer_extents, size)

        # Define base directory
        base_dir = "02_data_acquisition/tif_download_correctly_labelled_service_stations"
        
        # Ensure the directory exists
        os.makedirs(base_dir, exist_ok=True)

        # Save map as png
        file_path_png = f"02_data_acquisition/tif_download_correctly_labelled_service_stations/{id_rest}_{name}.png"
        download_as_png(response, file_path_png)

        # Safe tif from png
        file_path_tif = f"02_data_acquisition/tif_download_correctly_labelled_service_stations/{id_rest}_{name}.tif"
        download_tif_from_png(file_path_png, file_path_tif, bbox)

        # Delete png
        delete_file(file_path_png)

        print(f"\nDownload completed for image {index}: {name} ({id_rest})")
        print("----------------------------------------------------------------------------------------- \n")

In [166]:
mass_download_tif(correctly_labelled_service_stations_bbox, wms_urls, layer_extents, size = 2560)


WMS_NI_DOP does not contain bounding box.
bebb_dop20c does not contain bounding box.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 does not contain bounding box.
he_dop_rgb returns image.


/opt/homebrew/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)



Download completed for image 0: Schäferborn (lon_8.6630029_lat_50.2542348)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP does not contain bounding box.
bebb_dop20c does not contain bounding box.
th_dop returns image.

Download completed for image 1: Habichtsfang (lon_11.3531491_lat_50.9378362)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns image.

Download completed for image 2: Roddau (lon_10.2712414_lat_53.331074)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns blank image.
bebb_dop20c does not contain bounding box.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 does not contain bounding box.
he_dop_rgb does not contain bounding box.
sh_dop20_rgb returns image.

Download completed for image 3: Steinburg (lon_9.6300119_lat_53.8303266)
------------------------------------------------------

ValueError: No layer intersects with the provided bounding box.

Kapellenberg (lon_12.2060105_lat_51.4912736)(index 232) did not return an image, as it is located at the border of WMS services. Restarted download from index 233:

In [186]:
correctly_labelled_service_stations_bbox_rest = correctly_labelled_service_stations_bbox[233:]

In [188]:
mass_download_tif(correctly_labelled_service_stations_bbox_rest, wms_urls, layer_extents, size = 2560)


WMS_NI_DOP returns blank image.
bebb_dop20c does not contain bounding box.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 does not contain bounding box.
he_dop_rgb does not contain bounding box.
sh_dop20_rgb returns image.


/opt/homebrew/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)



Download completed for image 260: Depenauer Moor (lon_10.2127333_lat_54.1387697)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns image.

Download completed for image 261: None (lon_8.601339_lat_53.0351727)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns image.

Download completed for image 262: Rastplatz Hemmelsberg (lon_8.3213798_lat_53.0763179)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns image.

Download completed for image 263: None (lon_8.3918309_lat_53.0580378)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP does not contain bounding box.
bebb_dop20c does not contain bounding box.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 does not contain bounding box.
he_dop_rgb returns image.

Download completed for image 264: Ras

## Mass-Download correctly laballed service stations as tifs

In [226]:
def mass_download_tif(geodataframe, wms_urls, layer_extents, size = 2560):
    """
    Download a large number of GeoTIFF images from WMS services

    Parameters:
    geodataframe (GeoDataFrame): GeoDataFrame containing the bounding boxes
    wms_urls (dict): dictionary containing layer_names as keys and urls as values
    layer_extents: dictionary of layer names and their extent bounding boxes
    size (int): determines the size of the output png in pixels
    """

    for index, row in geodataframe.iterrows():
        bbox = row["bbox"]
        id_rest = row["id_rest"]
        name = row["name"]

        print(f"Downloading image {index}: {name} ({id_rest})\n")

        # Create map
        response = create_png_from_bbox(bbox, wms_urls, layer_extents, size)

        # Define base directory
        base_dir = "02_data_acquisition/tif_download_wrongly_labelled_service_stations"
        
        # Ensure the directory exists
        os.makedirs(base_dir, exist_ok=True)

        # Save map as png
        file_path_png = f"02_data_acquisition/tif_download_wrongly_labelled_service_stations/{id_rest}_{name}.png"
        download_as_png(response, file_path_png)

        # Safe tif from png
        file_path_tif = f"02_data_acquisition/tif_download_wrongly_labelled_service_stations/{id_rest}_{name}.tif"
        download_tif_from_png(file_path_png, file_path_tif, bbox)

        # Delete png
        delete_file(file_path_png)

        print(f"\nDownload completed for image {index}: {name} ({id_rest})")
        print("----------------------------------------------------------------------------------------- \n")

In [227]:
mass_download_tif(wrongly_labelled_service_stations_bbox, wms_urls, layer_extents, size = 2560)


WMS_NI_DOP returns blank image.
bebb_dop20c returns blank image.
th_dop returns image.


/opt/homebrew/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)



Download completed for image 0: Hohe Schrecke West (lon_11.2442867_lat_51.3248863)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP does not contain bounding box.
bebb_dop20c returns image.

Download completed for image 1: Probstheide (lon_13.5458137_lat_52.7129474)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP does not contain bounding box.
bebb_dop20c returns image.

Download completed for image 2: Ladeburger Heide (lon_13.549042_lat_52.7092396)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP does not contain bounding box.
bebb_dop20c does not contain bounding box.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 does not contain bounding box.
he_dop_rgb returns image.

Download completed for image 3: Spießwald (lon_8.6624068_lat_50.2544468)
------------------------------------------------------------------

ValueError: No layer intersects with the provided bounding box.

Fuchsaue Nord (lon_12.2147233_lat_51.2415014) does not return an image.

In [237]:
wrongly_labelled_service_stations_bbox_rest = wrongly_labelled_service_stations_bbox[17:]

In [238]:
mass_download_tif(wrongly_labelled_service_stations_bbox_rest, wms_urls, layer_extents, size = 2560)


WMS_NI_DOP returns image.


/opt/homebrew/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)



Download completed for image 17: Humboldtblick (lon_9.6690837_lat_51.3960842)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns image.

Download completed for image 18: Herkulesblick (lon_9.5648832_lat_51.3206232)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns image.

Download completed for image 19: Hedemünden (lon_9.7765986_lat_51.39495)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns blank image.
bebb_dop20c does not contain bounding box.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 does not contain bounding box.
he_dop_rgb returns image.

Download completed for image 20: Brasselberg (lon_9.4159049_lat_51.2726438)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns blank image.
bebb_dop20c does not contain bounding box.
t

Fuchsaue Nord (lon_12.2147233_lat_51.2415014) did not return an image.

- lon_8.6579523_lat_53.6066306_Bütteler Holz Ost
- lon_8.6591991_lat_53.6100326_Bütteler Holz West
- lon_8.9424542_lat_53.0280869_Mahndorfer Marsch
- lon_9.5648832_lat_51.3206232_Herkulesblick
- lon_10.6579729_lat_51.9212477_Brockenblick
- lon_10.6630829_lat_51.917356_Brockenblick

returned faulty images. Large swathes of the images at WMS borders.

Hence, they were deleted from the folder, resulting in 201 images.

# Delete service station from gpd with missing image

## Correctly labelled service stations

Kapellenberg (lon_12.2060105_lat_51.4912736)has returned no image, as this service station is located at the borders of WMS services.

In [27]:
# Load the GeoJSON files of correctly rest stops
parking_at_correct = gpd.read_file("02_data_acquisition/verified_parking_data/parking/parking_at_correct.geojson") # polygons of parkings areas at correctly labelled service stations
correctly_labelled_service_stations = gpd.read_file("02_data_acquisition/verified_parking_data/service_stations/correctly_labelled_service_stations.geojson") # polygons of correctly labelled service stations

# Load bbox csv
correctly_labelled_service_stations_bbox = pd.read_csv("02_data_acquisition/verified_parking_data/bounding_boxes/correctly_labelled_service_stations_bbox.csv")

In [28]:
# Check shapes of dataframes

print("Shape of parking_at_correct before dropping:", parking_at_correct.shape)
print("Shape of correctly_labelled_service_stations before dropping:", correctly_labelled_service_stations.shape)
print("Shape of correctly_labelled_service_stations_bbox before dropping:", correctly_labelled_service_stations_bbox.shape, "\n")

# Drop rows referring to Kapellenberg (lon_12.2060105_lat_51.4912736) by id_rest
correctly_labelled_service_stations = correctly_labelled_service_stations[~correctly_labelled_service_stations["id_rest"].isin(["lon_12.2060105_lat_51.4912736"])]
correctly_labelled_service_stations_bbox = correctly_labelled_service_stations_bbox[~correctly_labelled_service_stations_bbox["id_rest"].isin(["lon_12.2060105_lat_51.4912736"])]
parking_at_correct = parking_at_correct[~parking_at_correct["id_rest"].isin(["lon_12.2060105_lat_51.4912736"])]

print("Shape of parking_at_correct after dropping:", parking_at_correct.shape)
print("Shape of correctly_labelled_service_stations after dropping:", correctly_labelled_service_stations.shape)
print("Shape of correctly_labelled_service_stations_bbox after dropping:", correctly_labelled_service_stations_bbox.shape)


Shape of parking_at_correct before dropping: (1304, 4)
Shape of correctly_labelled_service_stations before dropping: (296, 4)
Shape of correctly_labelled_service_stations_bbox before dropping: (296, 4) 

Shape of parking_at_correct after dropping: (1300, 4)
Shape of correctly_labelled_service_stations after dropping: (295, 4)
Shape of correctly_labelled_service_stations_bbox after dropping: (295, 4)


In [29]:
correctly_labelled_service_stations_bbox

,id_rest,id_OSM_rest,name,bbox
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"[8.662897964530508, 50.24947031826926, 8.66990..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"[11.352126356256656, 50.93673132065365, 11.359..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"[10.264345359109937, 53.328233742563924, 10.27..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"[9.629200346595411, 53.82616873812249, 9.63679..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"[9.628319773449437, 53.82730601524238, 9.63591..."
...,...,...,...,...
291,lon_12.7580653_lat_52.8778128,way/915630046,Am Rhinluch,"[12.754311642753441, 52.87373430027324, 12.761..."
292,lon_7.2005154_lat_52.2816225,way/988014767,Gut Friedrichstal,"[7.196198912998569, 52.27916482977726, 7.20352..."
293,lon_8.5597202_lat_49.5969765,way/992004892,Wildbahn,"[8.554638173536537, 49.59453943341597, 8.56155..."
294,lon_10.7306355_lat_50.559048,way/1171916133,Adlersberg,"[10.724665021750997, 50.55768307908295, 10.731..."


In [30]:
# Download parking_at_correct, correctly_labelled_service_stations as GeoJSN and correctly_labelled_service_stations_bbox as csv

# Specify the path where you want to save the GeoJSON file
output_path_correctly_labelled_service_stations = "02_data_acquisition/verified_parking_data/service_stations/correctly_labelled_service_stations_post_download.geojson"
output_path_parking_at_correct = "02_data_acquisition/verified_parking_data/parking/parking_at_correct_post_download.geojson"
output_path_correctly_labelled_service_stations_bbox = "02_data_acquisition/verified_parking_data/bounding_boxes/correctly_labelled_service_stations_bbox_post_download.csv"

# Export rest_stations_ver_final and parking_areas_ver_final as GeoJSON
correctly_labelled_service_stations.to_file(output_path_correctly_labelled_service_stations, driver='GeoJSON')
parking_at_correct.to_file(output_path_parking_at_correct, driver='GeoJSON')

# Export rest_stations_ver_bbox_final to CSV
correctly_labelled_service_stations_bbox.to_csv(output_path_correctly_labelled_service_stations_bbox, index = False)

## Wrongly labelled service stations

In [32]:
# Load the GeoJSON files of wrongly labelled rest stops
parking_at_wrong = gpd.read_file("02_data_acquisition/verified_parking_data/parking/parking_at_wrong.geojson") # polygons of parkings areas at wrongly labelled service stations
wrongly_labelled_service_stations = gpd.read_file("02_data_acquisition/verified_parking_data/service_stations/wrongly_labelled_service_stations.geojson") # polygons of wrongly labelled service stations

# Load bbox csv
wrongly_labelled_service_stations_bbox = pd.read_csv("02_data_acquisition/verified_parking_data/bounding_boxes/wrongly_labelled_service_stations_bbox.csv")

Drop 

- Fuchsaue Nord (lon_12.2147233_lat_51.2415014)
- Bütteler Holz Ost (lon_8.6579523_lat_53.6066306)
- Bütteler Holz West (lon_8.6591991_lat_53.6100326)
- Mahndorfer Marsch (lon_8.9424542_lat_53.0280869)
- Herkulesblick(lon_9.5648832_lat_51.3206232)
- Brockenblick (lon_10.6579729_lat_51.9212477)
- Brockenblick (lon_10.6630829_lat_51.917356)

In [33]:
# Check shapes of dataframes

print("Shape of parking_at_wrong before dropping:", parking_at_wrong.shape)
print("Shape of wrongly_labelled_service_stations before dropping:", wrongly_labelled_service_stations.shape)
print("Shape of wrongly_labelled_service_stations_bbox before dropping:", wrongly_labelled_service_stations_bbox.shape, "\n")

# Drop rows of services stations with faulty images by id_rest
wrongly_labelled_service_stations = wrongly_labelled_service_stations[~wrongly_labelled_service_stations["id_rest"].isin(["lon_12.2147233_lat_51.2415014", "lon_8.6579523_lat_53.6066306", "lon_8.6591991_lat_53.6100326", "lon_8.9424542_lat_53.0280869", "lon_9.5648832_lat_51.3206232", "lon_10.6579729_lat_51.9212477", "lon_10.6630829_lat_51.917356"])]
wrongly_labelled_service_stations_bbox = wrongly_labelled_service_stations_bbox[~wrongly_labelled_service_stations_bbox["id_rest"].isin(["lon_12.2147233_lat_51.2415014", "lon_8.6579523_lat_53.6066306", "lon_8.6591991_lat_53.6100326", "lon_8.9424542_lat_53.0280869", "lon_9.5648832_lat_51.3206232", "lon_10.6579729_lat_51.9212477", "lon_10.6630829_lat_51.917356"])]
parking_at_wrong = parking_at_wrong[~parking_at_wrong["id_rest"].isin(["lon_12.2147233_lat_51.2415014", "lon_8.6579523_lat_53.6066306", "lon_8.6591991_lat_53.6100326", "lon_8.9424542_lat_53.0280869", "lon_9.5648832_lat_51.3206232", "lon_10.6579729_lat_51.9212477", "lon_10.6630829_lat_51.917356"])]

print("Shape of parking_at_wrong after dropping:", parking_at_wrong.shape)
print("Shape of wrongly_labelled_service_stations after dropping:", wrongly_labelled_service_stations.shape)
print("Shape of wrongly_labelled_service_stations_bbox after dropping:", wrongly_labelled_service_stations_bbox.shape)


Shape of parking_at_wrong before dropping: (299, 4)
Shape of wrongly_labelled_service_stations before dropping: (208, 4)
Shape of wrongly_labelled_service_stations_bbox before dropping: (208, 4) 

Shape of parking_at_wrong after dropping: (281, 4)
Shape of wrongly_labelled_service_stations after dropping: (201, 4)
Shape of wrongly_labelled_service_stations_bbox after dropping: (201, 4)


In [35]:
# Download parking_at_wrong, wrongly_labelled_service_stations as GeoJSN and wrongly_labelled_service_stations_bbox as csv

# Specify the path where you want to save the GeoJSON file
output_path_wrongly_labelled_service_stations = "02_data_acquisition/verified_parking_data/service_stations/wrongly_labelled_service_stations_post_download.geojson"
output_path_parking_at_wrong = "02_data_acquisition/verified_parking_data/parking/parking_at_wrong_post_download.geojson"
output_path_wrongly_labelled_service_stations_bbox = "02_data_acquisition/verified_parking_data/bounding_boxes/wrongly_labelled_service_stations_bbox_post_download.csv"

# Export rest_stations_ver_final and parking_areas_ver_final as GeoJSON
wrongly_labelled_service_stations.to_file(output_path_wrongly_labelled_service_stations, driver='GeoJSON')
parking_at_wrong.to_file(output_path_parking_at_wrong, driver='GeoJSON')

# Export rest_stations_ver_bbox_final to CSV
wrongly_labelled_service_stations_bbox.to_csv(output_path_wrongly_labelled_service_stations_bbox, index = False)